## Homework: Evaluation and Monitoring

In this homework, we'll evaluate the quality of our RAG system.

## Data

We'll evaluate the quality of our RAG system with [gpt-4o-mini](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv) - only the first 300 documents

In [1]:
import pandas as pd

github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)
df = df.iloc[:300]
print(len(df))
print(df.head(3))

300
                                          answer_llm  \
0  You can sign up for the course by visiting the...   
1  You can sign up using the link provided in the...   
2  Yes, there is an FAQ for the Machine Learning ...   

                                         answer_orig  document  \
0  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
1  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
2  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   

                                            question  \
0                Where can I sign up for the course?   
1                 Can you provide a link to sign up?   
2  Is there an FAQ for this Machine Learning course?   

                      course  
0  machine-learning-zoomcamp  
1  machine-learning-zoomcamp  
2  machine-learning-zoomcamp  


In [7]:
documents = df.to_dict(orient='records')
documents[0]

{'answer_llm': 'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).',
 'answer_orig': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

## Q1. Embeddings model

Get the embeddings model `multi-qa-mpnet-base-dot-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

In [3]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-mpnet-base-dot-v1'
model = SentenceTransformer(model_name)

/home/codespace/.python/current/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Create the embeddings for the first LLM answer.

What's the first value of the resulting vector?

In [5]:
answer_llm = df.iloc[0].answer_llm
print("answer_llm:", answer_llm)
embedding_llm = model.encode(answer_llm)
print("embedding_llm:", embedding_llm[0])

answer_llm: You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).
embedding_llm: -0.42244655


## Q2. Computing the dot product

Now for each answer pair, let's create embeddings and compute dot product between them

We will put the results (scores) into the `evaluations` list

What's the 75% percentile of the score?

* 21.67
* 31.67 <--
* 41.67
* 51.67

In [14]:
from tqdm.auto import tqdm

evaluations = []
for doc in tqdm(documents):
    answer_llm_embed = model.encode(doc['answer_llm'])
    answer_orig_embed = model.encode(doc['answer_orig'])
    dot_product = answer_llm_embed.dot(answer_orig_embed)
    evaluations.append(dot_product)

print(evaluations)

  0%|          | 0/300 [00:00<?, ?it/s]

[17.515987, 13.418402, 25.313255, 12.147415, 18.747736, 33.970406, 30.251705, 29.521576, 35.272198, 27.751772, 32.34471, 31.441843, 36.38072, 33.340504, 30.606163, 32.503044, 29.67445, 24.35346, 20.13247, 23.99548, 30.88028, 32.692436, 30.049173, 16.078167, 31.79642, 37.980003, 20.839046, 32.612865, 38.894196, 34.051826, 28.263878, 27.124832, 23.975264, 26.340149, 18.658115, 25.016403, 21.101131, 33.72679, 29.340347, 28.654505, 29.608585, 30.810738, 33.331203, 26.220482, 26.550072, 13.148597, 12.962547, 12.275609, 9.9744425, 10.883928, 29.845068, 32.36178, 22.18718, 30.268936, 25.091877, 32.742783, 28.220984, 27.274975, 24.208645, 22.568907, 19.767456, 18.679333, 20.422321, 22.051325, 18.188013, 28.455883, 25.919708, 23.33234, 22.20594, 28.296299, 39.23055, 36.758507, 31.913895, 31.202858, 36.913048, 30.514185, 36.26145, 27.397552, 37.792786, 23.29768, 34.252586, 34.550625, 30.316462, 35.703526, 31.012535, 35.45961, 35.075775, 35.429832, 29.88118, 30.037102, 31.247982, 29.893936, 28.52

In [9]:
import numpy as np
print(np.percentile(evaluations, 75))

31.67430877685547


## Q3. Computing the cosine

From Q2, we can see that the results are not within the [0, 1] range. It's because the vectors coming from this model are not normalized.

So we need to normalize them.

To do it, we 

* Compute the norm of a vector
* Divide each element by this norm

So, for vector `v`, it'll be `v / ||v||`


In [11]:
def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    return 0 if norm==0 else (v / norm)

Let's put it into a function and then compute dot product 
between normalized vectors. This will give us cosine similarity

What's the 75% cosine in the scores?
* 0.63
* 0.73
* 0.83 <--
* 0.93

In [15]:
evaluations_norm = []
for doc in tqdm(documents):
    answer_llm_embed_norm = normalize_vector(model.encode(doc['answer_llm']))
    answer_orig_embed_norm = normalize_vector(model.encode(doc['answer_orig']))
    dot_product = answer_llm_embed_norm.dot(answer_orig_embed_norm)
    evaluations_norm.append(dot_product)

print(evaluations_norm)

  0%|          | 0/300 [00:00<?, ?it/s]

[0.5067539, 0.38854873, 0.7185989, 0.33726627, 0.5217923, 0.83053213, 0.7462835, 0.6944061, 0.84688616, 0.65590763, 0.7779559, 0.78356636, 0.90468806, 0.80630296, 0.72759616, 0.7751896, 0.71516633, 0.5890557, 0.53322953, 0.5857593, 0.81232715, 0.83714426, 0.76611555, 0.43333992, 0.81558585, 0.92667866, 0.552616, 0.7622108, 0.9452982, 0.8478371, 0.7192839, 0.6864791, 0.6100939, 0.64910805, 0.48555, 0.6549567, 0.52971876, 0.84890294, 0.73956215, 0.76096815, 0.70153177, 0.7140965, 0.77817, 0.6202106, 0.62210196, 0.33472955, 0.3324926, 0.31343076, 0.25845352, 0.27644622, 0.77109647, 0.89201, 0.5712719, 0.7779895, 0.7033882, 0.8988763, 0.7822658, 0.69761264, 0.6318737, 0.5829771, 0.59635806, 0.5221753, 0.5993201, 0.65132016, 0.53131604, 0.761606, 0.6682948, 0.6511333, 0.66239053, 0.75467545, 0.89955723, 0.87245953, 0.75394404, 0.7211681, 0.8531313, 0.74570763, 0.85769904, 0.6625385, 0.91524327, 0.55959284, 0.8276353, 0.8465157, 0.74230355, 0.8715825, 0.7529516, 0.8018651, 0.8231076, 0.80075

In [13]:
print(np.percentile(evaluations_norm, 75))

0.8362348973751068


## Q4. Rouge

Now we will explore an alternative metric - the ROUGE score.  

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:

```bash
pip install rouge
```


In [16]:
!pip install rouge==1.0.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


Let's compute the ROUGE score between the answers at the index 10 of our dataframe (`doc_id=5170565b`)

```
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
```

There are three scores: `rouge-1`, `rouge-2` and `rouge-l`, and precision, recall and F1 score for each.

* `rouge-1` - the overlap of unigrams,
* `rouge-2` - bigrams,
* `rouge-l` - the longest common subsequence

What's the F score for `rouge-1`?
- 0.35
- 0.45 <--
- 0.55
- 0.65

In [17]:
documents[10]

{'answer_llm': "Yes, all sessions are recorded, so if you miss one, you won't miss anything. You can catch up on the content later. Additionally, you can submit your questions in advance for office hours, and those sessions are also recorded.",
 'answer_orig': 'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.',
 'document': '5170565b',
 'question': 'Are sessions recorded if I miss one?',
 'course': 'machine-learning-zoomcamp'}

In [18]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(documents[10]['answer_llm'], documents[10]['answer_orig'])[0]
print(scores)


{'rouge-1': {'r': 0.45454545454545453, 'p': 0.45454545454545453, 'f': 0.45454544954545456}, 'rouge-2': {'r': 0.21621621621621623, 'p': 0.21621621621621623, 'f': 0.21621621121621637}, 'rouge-l': {'r': 0.3939393939393939, 'p': 0.3939393939393939, 'f': 0.393939388939394}}


## Q5. Average rouge score

Let's compute the average between `rouge-1`, `rouge-2` and `rouge-l` for the same record from Q4

- 0.35 <--
- 0.45
- 0.55
- 0.65

In [20]:
avg = np.mean([scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']])
print(avg)

0.35490034990035496


## Q6. Average rouge score for all the data points

Now let's compute the score for all the records


rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']

Create a dataframe from them

What's the agerage `rouge_2` across all the records?
- 0.10
- 0.20 <--
- 0.30
- 0.40

In [36]:
def rouge_avg_score(scores):
    rouge_1 = scores['rouge-1']['f']
    rouge_2 = scores['rouge-2']['f']
    rouge_l = scores['rouge-l']['f']   

    return {'rouge_1': rouge_1, 'rouge_2': rouge_2, 'rouge_l': rouge_l} 

In [37]:
evaluations_rouge=[]
for doc in tqdm(documents):
    scores = rouge_scorer.get_scores(doc['answer_llm'], doc['answer_orig'])[0]
    avg_score = rouge_avg_score(scores)
    evaluations_rouge.append(avg_score)

print(evaluations_rouge)

  0%|          | 0/300 [00:00<?, ?it/s]

[{'rouge_1': 0.09523809178130524, 'rouge_2': 0.028169010918468917, 'rouge_l': 0.09523809178130524}, {'rouge_1': 0.12499999641113292, 'rouge_2': 0.05555555225694465, 'rouge_l': 0.09374999641113295}, {'rouge_1': 0.41558441095631643, 'rouge_2': 0.17777777313333343, 'rouge_l': 0.3896103849822905}, {'rouge_1': 0.2162162117421476, 'rouge_2': 0.047058819111419105, 'rouge_l': 0.18918918471512064}, {'rouge_1': 0.14207649881095297, 'rouge_2': 0.03389830142092829, 'rouge_l': 0.12021857531368524}, {'rouge_1': 0.43137254522106894, 'rouge_2': 0.2580645130072841, 'rouge_l': 0.43137254522106894}, {'rouge_1': 0.41269840791131274, 'rouge_2': 0.17948717488494426, 'rouge_l': 0.34920634441924925}, {'rouge_1': 0.30434782321361065, 'rouge_2': 0.1403508750384734, 'rouge_l': 0.26086956234404535}, {'rouge_1': 0.5172413747919143, 'rouge_2': 0.3243243199415632, 'rouge_l': 0.4137930989298454}, {'rouge_1': 0.34374999517578125, 'rouge_2': 0.21428570946712028, 'rouge_l': 0.34374999517578125}, {'rouge_1': 0.4545454495

In [38]:
df_rouge_score= pd.DataFrame(evaluations_rouge)
df_rouge_score.head()

,rouge_1,rouge_2,rouge_l
0,0.095238,0.028169,0.095238
1,0.125000,0.055556,0.093750
2,0.415584,0.177778,0.389610
3,0.216216,0.047059,0.189189
4,0.142076,0.033898,0.120219


In [39]:
df_rouge_score['rouge_2'].mean()

0.20696501983423318